In [31]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel

In [32]:
bqm=BinaryQuadraticModel('BINARY')

In [33]:
pumps=[0, 1, 2, 3]
time=[0, 1]
flow=[2, 7, 3, 8]
costs=[[36, 27], [56, 65], [48, 36], [52, 16]]
demand=20

In [34]:
x=[[f'P{p}_AM', f'P{p}_PM'] for p in pumps]
print(x)

[['P0_AM', 'P0_PM'], ['P1_AM', 'P1_PM'], ['P2_AM', 'P2_PM'], ['P3_AM', 'P3_PM']]


#### Objective $\sum_{p}^{pump}\sum_{t}^{time}cost_{p,t}\cdot x_{p, t}$

In [35]:
for p in pumps:
    for t in time:
        bqm.add_variable(x[p][t], costs[p][t])

#### Constraint 1: Each pump runs at least once per day $\sum_{t}^{time}x_{p, t}\ge1$

In [36]:
for p in pumps:
    c1=[(x[p][t], 1) for t in time]
    bqm.add_linear_inequality_constraint(c1, lb=1, ub=len(time), lagrange_multiplier=5, label='c1_pump_'+str(p))

#### Constraint 2: At most 3 pumps can run at a time $\sum_{p}^{pump}x_{p, t}-3\le0$

In [37]:
for t in time:
    c2=[(x[p][t], 1) for p in pumps]
    bqm.add_linear_inequality_constraint(c2, constant=-3, lagrange_multiplier=1, label='c2_time_'+str(t))

#### Constraint 3: Total flow satisfies daily demand $(\sum_{p}^{pump}\sum_{t}^{time}flow_{p, t}\cdot x_{p, t})-demand=0$

In [38]:
c3=[(x[p][t], flow[p]) for t in time for p in pumps]
bqm.add_linear_equality_constraint(c3, constant=-demand, lagrange_multiplier=28)

In [39]:
sampler=EmbeddingComposite(DWaveSampler())
samplest=sampler.sample(bqm, num_reads=1000)

In [44]:
sample=samplest.first.sample
total_flow=0
total_cost=0

In [46]:
print("\n\tAM\tPM")
for p in pumps:
    printout="P"+str(p)
    for t in time:
        printout+="\t"+str(sample[x[p][t]])
        total_flow+=sample[x[p][t]]*flow[p]
        total_cost+=sample[x[p][t]]*cost[p][t]
    print(printout)

print("Total flow:", total_flow)
print("Total cost:", total_cost)


	AM	PM
P0	0	1
P1	1	0
P2	0	1
P3	0	1
Total flow: 20
Total cost: 135
